In [ ]:
# Creates a list of all of the .fits files in a folder 
fitsFileList = glob.glob("/Users/tmzyece25/Desktop/Academia/Physics Project/01) 58.770302 -38.19514/*.fits")
# Same for text files
textFileList = glob.glob("/Users/tmzyece25/Desktop/Academia/Physics Project/01) 58.770302 -38.19514/*.txt")
for fitsFile in fitsFileList:
    print("File Path: ", fitsFile)
    print("File Name :", os.path.basename(fitsFile))

 
for textFile in textFileList:
    print("File Path: ", textFile)
    print("File Name :", os.path.basename(textFile))


File Path:  /Users/tmzyece25/Desktop/Academia/Physics Project/01) 58.770302 -38.19514/23AugICORE_0001_W2_mosaic-int.fits
File Name : 23AugICORE_0001_W2_mosaic-int.fits
File Path:  /Users/tmzyece25/Desktop/Academia/Physics Project/01) 58.770302 -38.19514/17AugICORE_0001_W1_mosaic-int.fits
File Name : 17AugICORE_0001_W1_mosaic-int.fits
File Path:  /Users/tmzyece25/Desktop/Academia/Physics Project/01) 58.770302 -38.19514/10Jan1.unc.fits
File Name : 10Jan1.unc.fits
File Path:  /Users/tmzyece25/Desktop/Academia/Physics Project/01) 58.770302 -38.19514/10Aug1.unc.fits
File Name : 10Aug1.unc.fits
File Path:  /Users/tmzyece25/Desktop/Academia/Physics Project/01) 58.770302 -38.19514/20JanICORE_0001_W2_mosaic-unc.fits
File Name : 20JanICORE_0001_W2_mosaic-unc.fits
File Path:  /Users/tmzyece25/Desktop/Academia/Physics Project/01) 58.770302 -38.19514/20JanICORE_0001_W1_mosaic-int.fits
File Name : 20JanICORE_0001_W1_mosaic-int.fits
File Path:  /Users/tmzyece25/Desktop/Academia/Physics Project/01) 58

In [ ]:
# Imports

import os
import requests
import glob
from io import BytesIO
import astropy
from astropy.io import fits
import pandas as pd
import os
import re
from datetime import datetime

In [ ]:
# Utility Functions

def makeFileList(filePath, fileType):
    filePathAndType = ""
    filePathAndType += filePath+ "/*."+ fileType
    # Creates a list of all one file type given a path 
    fileList = glob.glob(filePathAndType)
    return fileList

def fileListToString(fileList):
    for file in fileList:
        print("File Path:", file)
        print("File Name:", getFileName(file))

def getFileName(filePath):
    return os.path.basename(filePath)

In [ ]:
#Data Loading

"""
I thought that it would be a good idea to convert the text files to dataframes 
(two-dimensional labeled tables like an Excel sheet or SQL table) to assisst with
data manipulation
"""

def loadTxtRecords(txtFile):
    """
    Reads a WISE-style database .txt file into a clean pandas DataFrame.
    Automatically detects the header line and keeps only useful columns:
    fname, date_obs, and band. Index is reset to start from 0.
    """

    # Step 1: detect header line
    header_line = None
    with open(txtFile, "r") as f:
        for i, line in enumerate(f):
            if line.startswith("|in_row_id"):  # unique indicator for header line
                header_line = i
                break

    if header_line is None:
        raise ValueError("Header line not found in file.")

    # Step 2: skip metadata and header + type lines
    skiprows = header_line + 2  # skip header and type row

    # Step 3: read the actual data (space-delimited)
    df = pd.read_csv(
        txtFile,
        delim_whitespace=True,
        skiprows=skiprows,
        header=None,
        names=[
            "in_row_id", "in_ra", "in_dec", "scan_id", "scangrp", "frame_num",
            "band", "qual_frame", "moon_sep", "dtanneal", "saa_sep", "unixt",
            "date", "time", "fname"
        ]
    )

    # Step 4: drop rows with missing or invalid filenames
    df = df[df['fname'].str.endswith(".fits")]

    # Step 5: combine date and time into one column
    df['date_obs'] = df['date'] + ' ' + df['time']

    # Step 6: keep only useful columns
    df = df[['fname', 'date_obs', 'band']]

    # Step 7: reset index
    df = df.reset_index(drop=True)

    return df

In [ ]:
#Lightly tested, likely need's more work...

def parseFilename(filename):
    """
    Extracts available info from a FITS filename.
    Looks for filter (W1, W2, etc.) and optionally date if present.
    Returns a dictionary. Uses None when info is missing.
    """
    base = os.path.basename(filename).replace(".fits", "")  # keep original case

    # Look for filter keywords (case-insensitive)
    filter_part = None
    for f in ["W1", "W2", "W3", "W4"]:
        if f.lower() in base.lower():
            filter_part = f
            break

    # Extract a date like '10Aug', '24Jan', etc.
    # Match 1-2 digits followed immediately by 3 letters, case-insensitive
    date_match = re.search(r'(\d{1,2}[A-Za-z]{3})', base)
    date_part = date_match.group(1) if date_match else None

    return {
        "filter": filter_part,
        "date": date_part
    }



In [ ]:
def readFitsHeader(filepath):
    """
    Reads DATE and NUMFRMS from a FITS file header.
    
    Returns a dictionary:
    {
        'date_obs': datetime object or None,
        'numfrms': int or None
    }
    """
    with fits.open(filepath) as hdul:
        header = hdul[0].header
        date_str = header.get("DATE", None)
        numfrms = header.get("NUMFRMS", None)

        # Convert DATE string to datetime if possible
        date_obs = None
        if date_str is not None:
            try:
                # Typical format in FITS headers: 'YYYY-MM-DDThh:mm:ss.sss'
                date_obs = datetime.fromisoformat(date_str.replace('T', ' '))
            except ValueError:
                # fallback if format is different
                date_obs = date_str

    return {"date_obs": date_obs, "numfrms": numfrms}

#Rudimentary Test

fits_file = "/Users/tmzyece25/Desktop/Academia/Physics Project/01) 58.770302 -38.19514/20AugICORE_0001_W2_mosaic-unc.fits"

header_info = readFitsHeader(fits_file)

print("FITS File:", fits_file)
print("DATE from header:", header_info['date_obs'])
print("NUMFRMS from header:", header_info['numfrms'])


FITS File: /Users/tmzyece25/Desktop/Academia/Physics Project/01) 58.770302 -38.19514/20AugICORE_0001_W2_mosaic-unc.fits
DATE from header: 2025-05-07 01:15:40
NUMFRMS from header: 51


In [ ]:
def compareHeaderAndTxt(fname, header_info, txt_record):
    """
    Compares the DATE and NUMFRMS between FITS header and text record.
    Returns a list of mismatches.
    
    header_info: output from readFitsHeader()
    txt_record: a pandas Series with keys 'fname', 'date_obs', 'band', and optional 'subframes'
    """
    mismatches = []

    # Compare date_obs
    fits_date = header_info.get("date_obs", None)
    txt_date_str = txt_record.get("date_obs", None)
    txt_date = None

    if txt_date_str is not None:
        try:
            txt_date = datetime.fromisoformat(txt_date_str)
        except ValueError:
            pass  # fallback: leave as None

    if fits_date is not None and txt_date is not None:
        # Compare only date part (ignore time)
        if fits_date.date() != txt_date.date():
            mismatches.append("Date mismatch")
    elif fits_date is None or txt_date is None:
        mismatches.append("Date missing in FITS or TXT")

    # Compare NUMFRMS / subframes
    fits_numfrms = header_info.get("numfrms", None)
    txt_subframes = txt_record.get("subframes", None)
    
    if fits_numfrms is not None and txt_subframes is not None:
        if int(fits_numfrms) != int(txt_subframes):
            mismatches.append("NUMFRMS mismatch")
    elif fits_numfrms is None or txt_subframes is None:
        mismatches.append("NUMFRMS missing in FITS or TXT")

    return mismatches

txt_record = {
    "fname": "20AugICORE_0001_W2_mosaic-unc.fits",
    "date_obs": "2025-05-07 01:15:40",
    "band": "W2",
    "subframes": 51
}

fits_file = "/Users/tmzyece25/Desktop/Academia/Physics Project/01) 58.770302 -38.19514/20AugICORE_0001_W2_mosaic-unc.fits"
header_info = readFitsHeader(fits_file)

mismatches = compareHeaderAndTxt(fits_file, header_info, txt_record)
print("Mismatches:", mismatches)

Mismatches: []


In [ ]:
def compareHeaderAndTxt(fname, header_info, txt_record):
    """
    Compares the DATE and NUMFRMS between FITS header and text record.
    Returns a list of mismatches.
    """
    mismatches = []

    # Compare date_obs (from txt) with DATE in header
    if header_info["DATE"] is not None:
        if not str(txt_record["date_obs"]).startswith(header_info["DATE"][:10]):
            mismatches.append("Date mismatch")

    # Compare number of frames if available
    if header_info["NUMFRMS"] is not None:
        # txt_record may need a 'subframes' column; add if available
        if "subframes" in txt_record and int(header_info["NUMFRMS"]) != int(txt_record["subframes"]):
            mismatches.append("NUMFRMS mismatch")

    return mismatches

In [ ]:
#Function Calls (Testing df)

fitsFileList = makeFileList("/Users/tmzyece25/Desktop/Academia/Physics Project/01) 58.770302 -38.19514", "fits")
#fileListToString(fitsFileList)
textFileList = makeFileList("/Users/tmzyece25/Desktop/Academia/Physics Project/01) 58.770302 -38.19514", "txt")
#fileListToString(textFileList)

textFile_df = loadTxtRecords(textFileList[0])
print(textFile_df)

                       fname                 date_obs band
0   01907r049-w1-int-1b.fits  2019-01-09 01:42:39.575    1
1   01907r050-w1-int-1b.fits  2019-01-09 01:42:50.575    1
2   01913r025-w1-int-1b.fits  2019-01-09 06:25:24.715    1
3   01915r048-w1-int-1b.fits  2019-01-09 07:59:28.746    1
4   01915r050-w1-int-1b.fits  2019-01-09 07:59:39.746    1
5   01917r024-w1-int-1b.fits  2019-01-09 09:33:43.778    1
6   01917r025-w1-int-1b.fits  2019-01-09 09:33:54.782    1
7   01919r049-w1-int-1b.fits  2019-01-09 11:07:58.809    1
8   01919r050-w1-int-1b.fits  2019-01-09 11:08:09.809    1
9   01921r025-w1-int-1b.fits  2019-01-09 12:42:13.841    1
10  01921r026-w1-int-1b.fits  2019-01-09 12:42:24.845    1
11  01923r049-w1-int-1b.fits  2019-01-09 14:16:28.877    1
12  01925r025-w1-int-1b.fits  2019-01-09 15:50:43.908    1
13  01927r048-w1-int-1b.fits  2019-01-09 17:24:47.944    1
14  01927r049-w1-int-1b.fits  2019-01-09 17:24:58.944    1
15  01929r024-w1-int-1b.fits  2019-01-09 18:59:02.971   

/var/folders/pf/nh38xgvs0nndkbgjwfbcnmbm0000gp/T/ipykernel_27747/2780874122.py:31: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(


In [ ]:
# Folder path
folder_path = "/Users/tmzyece25/Desktop/Academia/Physics Project/01) 58.770302 -38.19514"

# 1. Get all FITS and TXT files
fits_files = makeFileList(folder_path, "fits")
txt_files = makeFileList(folder_path, "txt")

# 2. Load TXT records (assuming 1 TXT file)
txt_df = loadTxtRecords(txt_files[0])

# Optional: add a 'subframes' column if TXT doesn't have it
if 'subframes' not in txt_df.columns:
    txt_df['subframes'] = 1  # or adjust based on your TXT data if known

# 3. Loop over all FITS files
for fits_file in fits_files:
    # Parse filename
    parsed = parseFilename(fits_file)
    fits_fname = os.path.basename(fits_file)
    fits_date_from_name = parsed.get("date", None)
    fits_filter_from_name = parsed.get("filter", None)
    
    # Read header info
    header_info = readFitsHeader(fits_file)

    # Find matching TXT record by filename (or closest match)
    txt_match = txt_df[txt_df['fname'] == fits_fname]
    if len(txt_match) == 0:
        print(f"No matching TXT record found for {fits_fname}")
        continue
    txt_record = txt_match.iloc[0]

    # Compare header and TXT record
    mismatches = compareHeaderAndTxt(fits_file, header_info, txt_record)

    # Additional checks: compare date/filter from filename to TXT
    if fits_date_from_name and not str(txt_record['date_obs']).startswith(fits_date_from_name):
        mismatches.append("Filename date mismatch")
    if fits_filter_from_name and str(txt_record['band']) != fits_filter_from_name:
        mismatches.append("Filename filter mismatch")

    # Print results
    if len(mismatches) == 0:
        print(f"{fits_fname}: All checks passed :))))")
    else:
        print(f"{fits_fname}: Mismatches -> {mismatches}")


No matching TXT record found for 23AugICORE_0001_W2_mosaic-int.fits
No matching TXT record found for 17AugICORE_0001_W1_mosaic-int.fits


/var/folders/pf/nh38xgvs0nndkbgjwfbcnmbm0000gp/T/ipykernel_27747/2780874122.py:31: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(


No matching TXT record found for 10Jan1.unc.fits
No matching TXT record found for 10Aug1.unc.fits
No matching TXT record found for 20JanICORE_0001_W2_mosaic-unc.fits
No matching TXT record found for 20JanICORE_0001_W1_mosaic-int.fits
No matching TXT record found for 17AugICORE_0001_W2_mosaic-unc.fits
No matching TXT record found for 23AugICORE_0001_W1_mosaic-unc.fits
No matching TXT record found for 21AugICORE_0001_W1_mosaic-int.fits
No matching TXT record found for 10Aug2.int.fits
No matching TXT record found for 10Jan2.int.fits
No matching TXT record found for 22JanICORE_0001_W1_mosaic-unc.fits
No matching TXT record found for 22JanICORE_0001_W2_mosaic-int.fits
No matching TXT record found for 21AugICORE_0001_W2_mosaic-unc.fits
No matching TXT record found for 18AugICORE_0001_W2_mosaic-int.fits
No matching TXT record found for 11Jan1.unc.fits
No matching TXT record found for 16Jan1.unc.fits
No matching TXT record found for 16Aug1.unc.fits
No matching TXT record found for 18AugICORE_0